## Fonction de fusion des modèles

In [1]:
#Les modèles ont déjà été renommés
#On ajoute aux réactions de type EX_xxx le metabolite xxx_pool
#On ajoute la réaction xxx_pool <-> que l'on nomme "pool_xxx"
import cobra.test
from cobra import *

In [2]:
def FinderOfEX(model):
    '''
        Retrun a dictList of cobra reactions starting with EX
    '''
    res=[]
    for x in model.reactions:
        if type(x.id)==type("string") and x.id.startswith("EX"): #comparaison des types pour sauter les reactions.. 
                                                                #..sans noms : x.name de type NoneType.
            res.append(x.id)
    return res

In [53]:
def fusion(modelList):
    
    '''
    --Parameters--

    modelList est une liste de modèles. En pratique de longueur 2 ou 3
    
    --Return--
    Return the fusion of the models 
    '''
    
    from cobra import Model, Reaction, Metabolite
    #Initialisation
    NewModel = cobra.Model("Fusion des modèles !")
    
    #Ajout de toutes les réactions dans le nouveau modèle 
    
    for model in modelList:
        NewModel.add_reactions(model.reactions)
    
    
    #Modification des réactions EX comme décrit:
    EX_Reactions=[]
    for model in modelList:
        EX_Reactions+=FinderOfEX(model)
    
    #help(type(NewModel))
    for idEX in EX_Reactions:
        rootReac = NewModel.reactions.get_by_id(idEX)#Reaction avec une seule métabolite que l'on récupère:
        #print(NewModel.reactions)
        for metab in rootReac.metabolites:#Un seul métabolite #utilisation boucle car difficultés à récupérer autrement
            rootMetab = NewModel.metabolites.get_by_id(metab.id)
        poolMetab = Metabolite(
            'pool_'+rootMetab.id,
            formula=rootMetab.formula,
            compartment='pool',
            name='Version pool de '+rootMetab.name
                )
        rootReac.add_metabolites({ ##
            poolMetab : -rootMetab.charge #charge = coefficient stoechiometrique de root dans la reaction i.e. -1 normalement
        })
        #Ajout des réaction xxx_pool <-> que l'on identifient par "pool_xxx"
        poolReac = Reaction('pool_'+rootMetab.id)
        poolReac.name='Reaction : '+rootMetab.id+'_pool <->'
        poolReac.subsystem = 'pool'
        #Bounds will be set by the diet.
        poolReac.add_metabolites({
            poolMetab : -1.0
        })
        NewModel.add_reactions([poolReac])
    ##Changement de l'objectif ?
    #XXX
    return NewModel

## TESTS

In [5]:
mini=cobra.test.create_test_model("mini")

In [6]:
# Best practise: SBML compliant IDs
modeltest = Model('example_model')

reaction = Reaction('REACTION_INSEREE')
reaction.name = 'reaction que jai insérée '
reaction.subsystem = 'FAUDRA FAIRE GAFFE AU SUBSYSTEM'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [7]:
TEST = Metabolite(
    'TEST',
    formula='FORMULETEST',
    name='NAMETEST',
    compartment='test')
reaction.add_metabolites({
    TEST : -1.0 
})


In [8]:
modeltest.add_reactions([reaction])

In [9]:
modelList=[modeltest,mini]

In [63]:
fusion(modelList)

Name,Fusion des modèles !
Memory address,0x07f31a01ffe80
Number of metabolites,27
Number of reactions,22
Objective expression,0
Compartments,"test, c, e, pool"


La fusion à l'air de fonctionner, il faudra faire des tests plus approfondis certainement.